# PyDipAPI - Batch-Operationen und Caching

In diesem Notebook lernen Sie fortgeschrittene Funktionen fuer effizientes Arbeiten.

## Inhalt:
1. Batch-Anfragen fuer mehrere IDs
2. Caching aktivieren und nutzen
3. Performance-Optimierung
4. Best Practices

In [ ]:
# Setup mit Caching
import sys
import os
import time
sys.path.insert(0, os.path.abspath('..'))

from pydipapi import DipAnfrage

# WICHTIG: Ersetzen Sie dies mit Ihrem echten API-Schluessel!
API_KEY = "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN"

if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    # Client mit aktiviertem Caching
    client = DipAnfrage(
        api_key=API_KEY,
        enable_cache=True,
        cache_ttl=3600,
        rate_limit_delay=0.2
    )
    print("Client mit Caching initialisiert!")
else:
    print("BITTE ERSETZEN SIE DEN PLATZHALTER MIT IHREM ECHTEN API-SCHLUESSEL!")
    print("Den API-Schluessel erhalten Sie unter: https://dip.bundestag.de/api/")

## Batch-Anfragen

Batch-Anfragen erlauben es, mehrere IDs auf einmal abzufragen. Das ist viel effizienter als einzelne Anfragen:

In [ ]:
# Batch-Anfrage demonstrieren
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        print("Sammle IDs fuer Batch-Anfragen...")
        
        # Sammle einige IDs
        docs = client.get_drucksache(anzahl=5)
        doc_ids = [doc.get('id') for doc in docs if doc.get('id')][:3]
        
        print(f"{len(doc_ids)} Dokument-IDs gesammelt: {doc_ids}")
        
        # Batch-Anfrage
        if doc_ids:
            start_time = time.time()
            batch_docs = client.get_drucksache_ids(doc_ids)
            batch_time = time.time() - start_time
            
            print(f"Batch-Anfrage in {batch_time:.2f}s abgeschlossen")
            print(f"{len(batch_docs)} Dokumente erhalten")
            
            if batch_docs:
                print(f"\nBeispiel: {batch_docs[0].get('titel', 'N/A')[:60]}...")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

## Vergleich: Einzelanfragen vs. Batch-Anfragen

Lassen Sie uns die Performance vergleichen:

In [ ]:
# Performance-Vergleich
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        # IDs sammeln
        persons = client.get_person(anzahl=5)
        person_ids = [p.get('id') for p in persons if p.get('id')][:3]
        
        if person_ids:
            print(f"Vergleiche Performance mit {len(person_ids)} IDs\n")
            
            # Einzelanfragen
            print("1. Einzelanfragen:")
            start_time = time.time()
            individual_results = []
            for pid in person_ids:
                person = client.get_person_id(pid)
                if person:
                    individual_results.append(person)
            individual_time = time.time() - start_time
            print(f"   Zeit: {individual_time:.2f}s")
            print(f"   Ergebnisse: {len(individual_results)}")
            
            # Batch-Anfrage
            print("\n2. Batch-Anfrage:")
            start_time = time.time()
            batch_results = client.get_person_ids(person_ids)
            batch_time = time.time() - start_time
            print(f"   Zeit: {batch_time:.2f}s")
            print(f"   Ergebnisse: {len(batch_results)}")
            
            # Vergleich
            if batch_time > 0:
                speedup = individual_time / batch_time
                print(f"\nBatch-Anfrage ist {speedup:.1f}x schneller!")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

## Caching verstehen

Caching speichert API-Antworten lokal, um wiederholte Anfragen zu vermeiden:

In [ ]:
# Caching demonstrieren
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    try:
        print("Teste Caching-Verhalten...\n")
        
        # Erste Anfrage (wird gecacht)
        print("1. Erste Anfrage (wird gecacht):")
        start_time = time.time()
        docs1 = client.get_drucksache(anzahl=3)
        time1 = time.time() - start_time
        print(f"   Zeit: {time1:.2f}s")
        print(f"   Dokumente: {len(docs1)}")
        
        # Zweite Anfrage (sollte aus Cache kommen)
        print("\n2. Zweite Anfrage (aus Cache):")
        start_time = time.time()
        docs2 = client.get_drucksache(anzahl=3)
        time2 = time.time() - start_time
        print(f"   Zeit: {time2:.2f}s")
        print(f"   Dokumente: {len(docs2)}")
        
        if time2 < time1:
            speedup = time1 / time2 if time2 > 0 else 0
            print(f"\nCache ist {speedup:.1f}x schneller!")
    except Exception as e:
        print(f"Fehler: {e}")
else:
    print("Bitte API-Schluessel konfigurieren.")

## Caching konfigurieren

Sie koennen Caching-Einstellungen anpassen:

In [ ]:
# Verschiedene Caching-Konfigurationen
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    print("Caching-Konfigurationen:\n")
    
    # Client ohne Caching
    client_no_cache = DipAnfrage(
        api_key=API_KEY,
        enable_cache=False
    )
    print("1. Client ohne Caching:")
    print("   enable_cache=False")
    
    # Client mit kurzer TTL (1 Stunde)
    client_short_ttl = DipAnfrage(
        api_key=API_KEY,
        enable_cache=True,
        cache_ttl=3600  # 1 Stunde
    )
    print("\n2. Client mit kurzer TTL:")
    print("   enable_cache=True, cache_ttl=3600 (1 Stunde)")
    
    # Client mit langer TTL (24 Stunden)
    client_long_ttl = DipAnfrage(
        api_key=API_KEY,
        enable_cache=True,
        cache_ttl=86400  # 24 Stunden
    )
    print("\n3. Client mit langer TTL:")
    print("   enable_cache=True, cache_ttl=86400 (24 Stunden)")
    
    print("\nHinweis: TTL = Time To Live (Gültigkeitsdauer des Caches)")
else:
    print("Bitte API-Schluessel konfigurieren.")

## Rate-Limiting

Rate-Limiting verhindert, dass Sie zu viele Anfragen in kurzer Zeit stellen:

In [ ]:
# Rate-Limiting konfigurieren
if API_KEY != "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN":
    print("Rate-Limiting-Konfigurationen:\n")
    
    # Standard (0.1 Sekunden zwischen Anfragen)
    client_default = DipAnfrage(
        api_key=API_KEY,
        rate_limit_delay=0.1
    )
    print("1. Standard Rate-Limiting:")
    print("   rate_limit_delay=0.1 (100ms zwischen Anfragen)")
    
    # Konservativ (0.5 Sekunden)
    client_conservative = DipAnfrage(
        api_key=API_KEY,
        rate_limit_delay=0.5
    )
    print("\n2. Konservatives Rate-Limiting:")
    print("   rate_limit_delay=0.5 (500ms zwischen Anfragen)")
    
    print("\nHinweis: Groessere Delays schonen die API, sind aber langsamer")
else:
    print("Bitte API-Schluessel konfigurieren.")

## Best Practices

Wichtige Tipps fuer optimale Performance:

In [ ]:
# Best Practices
print("Performance-Best Practices:\n")
print("1. Batch-Anfragen nutzen:")
print("   client.get_drucksache_ids([1,2,3,4,5])  # Effizient")
print("   # Statt: 5 einzelne get_drucksache_by_id() Aufrufe\n")
print("2. Caching aktivieren:")
print("   client = DipAnfrage(api_key=key, enable_cache=True)")
print("   # Speichert Antworten lokal, vermeidet wiederholte Anfragen\n")
print("3. Rate-Limiting beachten:")
print("   client = DipAnfrage(api_key=key, rate_limit_delay=0.2)")
print("   # Verhindert API-Ueberlastung\n")
print("4. Spezifische Filter verwenden:")
print("   client.search_documents('term', wahlperiode=20, anzahl=50)")
print("   # Reduziert Datenmenge und verbessert Performance\n")
print("5. Angemessene anzahl-Parameter:")
print("   # Nicht zu gross waehlen (z.B. 100 statt 10000)\n")
print("6. Caching-TTL anpassen:")
print("   # Kurze TTL fuer haeufig aendernde Daten")
print("   # Lange TTL fuer statische Daten")

## Zusammenfassung

In diesem Notebook haben Sie gelernt:

- **Batch-Anfragen**: Mehrere IDs auf einmal abrufen (`get_*_ids()`)
- **Caching**: Lokale Speicherung von API-Antworten
- **Performance-Optimierung**: Batch vs. Einzelanfragen
- **Rate-Limiting**: Kontrolle der Anfragefrequenz
- **Best Practices**: Effiziente Nutzung der API

### Naechste Schritte

Weiter mit **Notebook 04**: Content-Parser fuer strukturierte Datenanalyse!